In [1]:
import os
import io
import ast
import boto3
import sagemaker
import numpy as np
import pandas as pd

from sagemaker.pytorch import PyTorch

from ibov.utils import load_config
from ibov.deploy import get_deploy_config, build_endpoint, kill_endpoint

### Loading Configs

In [2]:
# Load config dict
config = load_config()

In [3]:
role = config.get("sagemaker").get("role")
prefix = config.get("sagemaker").get("bucket_prefx")
data_dir = config.get("data").get("dir")
region = config.get("sagemaker").get("region")

In [4]:
dropout = config.get("model").get("dropout")
window = config.get("feature").get("window")
hidden_layer = config.get("model").get("hidden_layer")
lr = config.get("model").get("lr")
seed = config.get("model").get("seed")
epochs = config.get("model").get("epochs")

### Upload Data

In [5]:
session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

In [6]:
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [7]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibov", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge',
                    hyperparameters = {
                        "batch-size": 50,
                        "epochs": epochs,
                        "seed": seed,
                        "input-layer": window,
                        "hidden-layer": hidden_layer,
                        "dropout": dropout        
                    })

In [8]:
estimator.fit({'train': input_data, "config": input_config})

2021-01-25 19:12:35 Starting - Starting the training job...
2021-01-25 19:13:07 Starting - Launching requested ML instancesProfilerReport-1611601952: InProgress
.........
2021-01-25 19:14:49 Starting - Preparing the instances for training......
2021-01-25 19:15:50 Downloading - Downloading input data...
2021-01-25 19:16:30 Training - Downloading the training image...
2021-01-25 19:17:11 Uploading - Uploading generated training model
2021-01-25 19:17:11 Completed - Training job completed
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-25 19:16:53,295 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-25 19:16:53,323 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-25 19:16:53,327 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-25 19:16:53,618 sagemaker-containers INFO     Module train does n

Training seconds: 88
Billable seconds: 88


In [1]:
estimator.create_model()

NameError: name 'estimator' is not defined

### Endpoint Deployment

config = load_config()

deploy_config = get_deploy_config(config)

build_endpoint(deploy_config)

kill_endpoint(config)